In [6]:
import pandas as pd
import numpy as np
# https://www.kaggle.com/code/carlmcbrideellis/feature-selection-using-the-boruta-shap-package
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

In [2]:
data = pd.read_csv("data_with_new_features.csv")

In [3]:
data.isnull().sum()

loan_id                         0
age                             0
income                          0
loan_amount                     0
credit_score                    0
months_employed                 0
num_credit_lines                0
interest_rate                   0
loan_term                       0
dtiratio                        0
education                       0
employment_type                 0
marital_status                  0
has_mortgage                    0
has_dependents                  0
loan_purpose                    0
has_co_signer                   0
default                         0
income_to_loan_ratio            0
empl_sta                        0
loan_pay_to_inc_ratio           0
credit_util_ratio               0
credit_age_factor               0
interest_payment_burden         0
income_to_interest_ratio        0
credit_income_inter             0
high_dti_flag                   0
marital_status_dependents       0
employment_type_loan_purpose    0
dtype: int64

In [5]:
data_new = data.copy()

Since there is a high level of class imbalance in the target variable for <code>default</code>, we will use the <code>stratify</code> option while splitting the data into train, validation, and test sets.

In [9]:
y = data_new["default"]
del data_new["default"]

In [14]:
df_full_train, df_test, y_full_train, y_test = train_test_split(data_new, y, stratify = y, test_size = 0.2, random_state = 1)

df_train, df_val, y_train, y_val = train_test_split(df_full_train, y_full_train, stratify =y_full_train,  test_size = 0.25, random_state = 1)

In [17]:
df_full_train = df_full_train.reset_index(drop = True)
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [18]:
y_full_train = y_full_train.values
y_train = y_train.values
y_val = y_val.values
y_test = y_test.values

In [19]:
full_train_dicts = df_full_train.to_dict(orient = 'records')
train_dicts = df_train.to_dict(orient = 'records')
val_dicts = df_val.to_dict(orient = 'records')
test_dicts = df_test.to_dict(orient = 'records')

In [20]:
#I will apply one-hot encoding to the dataset

In [41]:
# I was getting a memory error, so I set sparse=True for the DictVectorizer.
dv = DictVectorizer(sparse = True)
dv.fit(train_dicts)
#X_full_train_new = dv.transform(full_train_dicts)
X_train_new = dv.transform(train_dicts)
X_val_new = dv.transform(val_dicts)
X_test_new = dv.transform(test_dicts)

ValueError: 
All the 30 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\linear_model\_logistic.py", line 1223, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-package

In [46]:
# Create a pipeline with DictVectorizer and RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
pipeline = Pipeline([
    ('vectorizer', DictVectorizer(sparse=True)),  # Convert categorical features into a sparse matrix
    ('classifier', LogisticRegression(max_iter=1000,class_weight='balanced')) # Random Forest classifier
])


In [47]:

# Hyperparameter grid to search
param_grid = {
    'classifier__C': [0.1, 1, 10],                # Regularization strength (lower values = stronger regularization)
    'classifier__solver': ['liblinear', 'saga'],  # Solvers to use for optimization
    'classifier__penalty': ['l2', 'elasticnet'],  # Type of regularization
    'classifier__max_iter': [100, 200, 500]       # Maximum number of iterations for optimization
}

In [49]:
# Perform GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
grid_search_lr =RandomizedSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)
# Fit the model
grid_search_lr.fit(full_train_dicts, y_full_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer', DictVectorizer()),
                                             ('classifier',
                                              LogisticRegression(class_weight='balanced',
                                                                 max_iter=1000))]),
                   n_jobs=-1,
                   param_distributions={'classifier__C': [0.1, 1, 10],
                                        'classifier__max_iter': [100, 200, 500],
                                        'classifier__penalty': ['l2',
                                                                'elasticnet'],
                                        'classifier__solver': ['liblinear',
                                                               'saga']},
                   verbose=1)

In [54]:
df_randomsearchcv_summary = grid_search_lr.score_summary()

AttributeError: 'RandomizedSearchCV' object has no attribute 'score_summary'

In [51]:
print("Best parameters:", grid_search_lr.best_params_)

Best parameters: {'classifier__solver': 'liblinear', 'classifier__penalty': 'l2', 'classifier__max_iter': 100, 'classifier__C': 0.1}


__Best parameters: {'classifier__solver': 'liblinear', 'classifier__penalty': 'l2', 'classifier__max_iter': 100, 'classifier__C': 0.1}__


In [80]:
from sklearn.ensemble import RandomForestClassifier


In [100]:
pipeline = Pipeline([
    ('vectorizer', DictVectorizer(sparse=True)),  # Convert categorical features into a sparse matrix
    ('classifier', RandomForestClassifier()) # Random Forest classifier
])


In [103]:

param_grid_rf = {
    'classifier__n_estimators': [100, 200, 300, 400, 500],          # Number of trees in the forest (higher = more accurate but slower)
    'classifier__max_depth': [None, 10, 20, 30, 40, 50],             # Max depth of the trees (None means nodes are expanded until all leaves are pure)
    'classifier__min_samples_split': [2, 5, 10],                     # Minimum samples required to split an internal node
    'classifier__min_samples_leaf': [1, 2, 4],                       # Minimum samples required at a leaf node
    'classifier__max_features': ['auto', 'sqrt', 'log2'],            # Number of features to consider when looking for the best split
    'classifier__bootstrap': [True, False],                          # Whether to use bootstrap sampling (True is most common)
    'classifier__class_weight': ['balanced'],                  # Use balanced weights to deal with imbalanced classes
    'classifier__max_samples': [None, 0.8, 0.9],                     # Number of samples to train each tree (only when bootstrap=True)
    'classifier__oob_score': [True, False],                          # Out-of-bag score to get unbiased estimate of model performance
    'classifier__random_state': [1]                                 # For reproducibility
}

In [106]:
random_search_rf =RandomizedSearchCV(pipeline, param_grid_rf, cv=5, n_jobs=-1, verbose=1, n_iter = 20)
# Fit the model
random_search_rf.fit(full_train_dicts, y_full_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
50 fits failed out of a total of 100.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Kuzey\Anaconda3\envs\ml-zoomcamp\Lib\site-packages

RandomizedSearchCV(cv=5,
                   estimator=Pipeline(steps=[('vectorizer', DictVectorizer()),
                                             ('classifier',
                                              RandomForestClassifier())]),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'classifier__bootstrap': [True, False],
                                        'classifier__class_weight': ['balanced'],
                                        'classifier__max_depth': [None, 10, 20,
                                                                  30, 40, 50],
                                        'classifier__max_features': ['auto',
                                                                     'sqrt',
                                                                     'log2'],
                                        'classifier__max_samples': [None, 0.8,
                                                                    0.9],
                                        'classifier__min_samples_leaf': [1, 2,
                                                                         4],
                                        'classifier__min_samples_split': [2, 5,
                                                                          10],
                                        'classifier__n_estimators': [100, 200,
                                                                     300, 400,
                                                                     500],
                                        'classifier__oob_score': [True, False],
                                        'classifier__random_state': [1]},
                   verbose=1)

In [107]:
print("Best parameters:", random_search_rf.best_params_)

Best parameters: {'classifier__random_state': 1, 'classifier__oob_score': False, 'classifier__n_estimators': 500, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 1, 'classifier__max_samples': 0.9, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 40, 'classifier__class_weight': 'balanced', 'classifier__bootstrap': True}



print("Best parameters:", random_search_rf.best_params_)
Best parameters: {'classifier__random_state': 1, 'classifier__oob_score': False, 'classifier__n_estimators': 500, 'classifier__min_samples_split': 10, 'classifier__min_samples_leaf': 1, 'classifier__max_samples': 0.9, 'classifier__max_features': 'sqrt', 'classifier__max_depth': 40, 'classifier__class_weight': 'balanced', 'classifier__bootstrap': True}